In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import xgboost as xg 
import matplotlib.pyplot as plt
from featuresv1 import features, target
%matplotlib inline

In [ ]:
data = pd.read_csv('merged_stat_score_data_clean.csv')
player_stats = pd.read_csv("player_stats.csv")

player_stats['season'] = [i.strftime('%Y') for i in pd.to_datetime(player_stats['utcStartTime']).to_list()]

In [ ]:
def generate_test_train_split(data, features, target, test_size=0.4):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [ ]:
def tpot_pipeline(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], list(data_['y_train'].score))
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [ ]:
tpot_pipeline(data, features, target)

In [ ]:
def fit_lin_regression(x_train, y_train):

    # fit linear regression line
    regr_model = LinearRegression()
    regr_model.fit(x_train, y_train)
    
    return regr_model

In [ ]:
def eval_lin_regression(x_test, y_test):
    # predict the values
    pred = regr.predict(x_test)
    
    # Regression Score 
    print("Regression Score (R-square): " + str(regr.score(x_test, y_test)))

    #RMSE
    print("RMSE: " + str(mean_squared_error(y_test, pred)**0.5))

In [ ]:
def fit_xgboost(x_train, y_train):# Instantiation 
    xgb_r = xg.XGBRegressor(objective ='reg:squarederror', 
                      n_estimators = 10, seed = 42) 

    # Fitting the model 
    xgb_r.fit(x_train, y_train)
    
    return xgb_r

In [ ]:
def eval_xgboost(x_test, y_test):
    # Predict the model 
    pred = xgb_r.predict(x_test) 

    # RMSE Computation 
    rmse = np.sqrt(mean_squared_error(y_test, pred)) 
    print("RMSE : % f" %(rmse)) 

In [ ]:
def return_stats_input(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
    if 'team.name' not in model_inputs:
        model_inputs.append('team.name')
        
    match_ups = data[(data['home.team.name']==home_team) & (data['away.team.name']==away_team)]
    group_by = ['season','round.roundNumber','team.name']
    team_stats = match_ups.groupby(group_by).sum()
    team_stats.reset_index(inplace=True)
    
    home_stats = team_stats[team_stats['team.name']==home_team][model_inputs].groupby('team.name').mean()
    
    away_stats = team_stats[team_stats['team.name']==away_team][model_inputs].groupby('team.name').mean()
    
    return (home_stats, away_stats)

In [ ]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [ ]:
def generate_tear_off_lr(model, round_matchups, player_stats,model_inputs):
    print('-------------------LINEAR REGRESSION TEAR-OFF-------------------')
    for tup in round_matchups:
        print("Home: " + tup[0] + '\t' + "Away: " + tup[1])
        print(f"{model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[0])}" + '\t\t\t' + \
             f"{model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[1])}")

In [ ]:
def generate_tear_off(model, round_matchups, player_stats,model_inputs):
    print('-------------------' + str(pipeline_optimizer.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        print("Home: " + tup[0] + '\t' + "Away: " + tup[1])
        print(f"{round(model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[0])[0])}" + '\t\t\t' + \
             f"{round(model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[1])[0])}")

In [ ]:
generate_tear_off(pipeline_optimizer, round_matchups, player_stats, features)

In [ ]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [ ]:
generate_tear_off(pipeline_optimizer, round1, player_stats, features)